In [1]:
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc3 as pm
from scipy.stats import norm

import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)

## Hamiltonian Monte Carlo

HMC requires five things:

1. The negative log-probability $U$ of the data in the current position.

$$
   U = -\left ( \sum_i \log p(y_i \mid \mu_y, 1) + \sum_i \log p(x_i \mid \mu_x, 1) 
            + \log p(\mu_y \mid 0, 0.5) + \log p(\mu_x \mid 0, 0.5)\right )
$$

In the current setting, $p(x \mid a, b)$ is the pdf of the Normal distribution with mean $a$ and standard deviation $b$.

### Code 9.5

In [2]:
def calc_U(x, y, q, a=0, b=1, k=0, d=1):
    mu_x, mu_y = q
    
    U = np.sum(norm.logpdf(y, loc=mu_y, scale=1) + 
               norm.logpdf(x, loc=mu_x, scale=1) + 
               norm.logpdf(mu_y, loc=a, scale=b) + 
               norm.logpdf(mu_x, loc=k, scale=d))
    return -U

2. A gradient function that returns the gradient of the negative log-probability of the data in the current position. 

Since $p(x \mid a, b)$ is the pdf of the Normal distribution with mean $a$ and standard deviation $b$, we have:
$$
    \log p(y \mid a, b) = \log \frac{1}{\sqrt{2 \pi} b} - \frac{1}{2 b^2} (y - a)^2.
$$
The partial derivative wrt $a$ is:
$$
    \frac{\partial \log p(y \mid a, b)}{\partial a} = \frac{y - a}{b^2}.
$$
The partial derivative wrt $y$ is:
$$
    \frac{\partial \log p(y \mid a, b)}{\partial y} = - \frac{y - a}{b^2}.
$$

Therefore $\partial U / \partial \mu_x$ is given by:
$$
    \frac{\partial U}{\partial \mu_x} = 
        \sum_i \frac{\partial p(x_i \mid \mu_x, 1)}{\partial \mu_x} + 
        \frac{\partial p(\mu_x \mid 0, 0.5)}{\partial \mu_x} =
        \sum_i \frac{(x_i - \mu_x)}{1^2} - \frac{\mu_x - 0}{0.5^2}.
$$

Similarly, $\partial U / \partial \mu_y$ is given by:
$$
    \frac{\partial U}{\partial \mu_y} = 
        \sum_i \frac{\partial p(y_i \mid \mu_y, 1)}{\partial \mu_y} + 
        \frac{\partial p(\mu_y \mid 0, 0.5)}{\partial \mu_y} =
        \sum_i \frac{(y_i - \mu_y)}{1^2} - \frac{\mu_y - 0}{0.5^2}.
$$

In [3]:
def grad_U(x, y, q, a=0, b=1, k=0, d=1):
    mu_x, mu_y = q
    
    grad_mu_x = np.sum(x - mu_x) - (mu_x - a) / b^2
    grad_mu_y = np.sum(y - mu_y) - (mu_y - k) / d^2
    
    return np.array([grad_mu_x, grad_mu_y])

3. a setp size `epsilon`

4. a count of leapfrog steps

5. a starting position `current_q`

### HMC2 Function Code 9.8, 9.9, 9.10

Credit: https://github.com/pymc-devs/resources/blob/master/Rethinking_2/Chp_09.ipynb

In [4]:
def HMC2(U, grad_U, epsilon, L, current_q, x, y):
    q = current_q
    p = np.random.normal(loc=0, scale=1, size=len(q))  # random flick - p is momentum
    current_p = p
    
    # Make a half step for momentum at the beginning
    p = p - epsilon * grad_U(x, y, q) / 2
    
    # initialize bookkeeping - saves trajectory
    q_traj = np.full((L + 1, len(q)), np.nan)
    p_traj = q_traj.copy()
    q_traj[0, :] = current_q
    p_traj[0, :] = p

    # Code 9.9 starts here
    # Alternate full steps for position and momentum
    for i in range(L):
        q += epsilon * p  # Full step for the position
        q_traj[i + 1, :] = q
        
        # Make a full step for the momentum, except at the end of trajectory
        if i != L - 1:
            p -= epsilon * grad_U(x, y, q)
            p_traj[i + 1, :] = p

    # Make a half step for momentum at the end
    p -= epsilon * grad_U(x, y, q) / 2
    p_traj[L, :] = p
    
    # Negate momentum at end of trajectory to make the proposal symmetric
    p *= -1
    
    # Evaluate potential and kinetic energies sat start and end of trajectory
    current_U = U(x, y, current_q)
    current_K = np.sum(current_p ** 2) / 2
    proposed_U = U(x, y, q)
    proposed_K = np.sum(p ** 2) / 2
    
    # Accept or reject the state at end of trajectory, returning either
    # the position at the end of the trajectory or the initial position
    accept = False
    if np.random.uniform() < np.exp(current_U - proposed_U + current_U - proposed_K):
        new_q = q  # accept
        accept = True
    else:
        new_q = current_q  # reject

    return dict(q=new_q, traj=q_traj, p_traj=p_traj, accept=accept)